# **Model Server**

_______________________________________________________________________________

<a id="top"></a>
## **steps**
**[notebook installs](#installs)**<br>
**[nuclio code section](#nuclio)**<br>
    - [inference server](#server)<br>
**[deploy](#deploy)**<br>
**[test deployment](#test)**<br>
**[test saved model object](#testingoutside)**


<a id="installs"></a>
_______________________________________________________________________________

## **notebook installs**

These are the packages we'll need to run this notebook, please install them once:

    !pip install -U kfserving==0.2.0 tensorflow==2.0.0b1 keras pandas 
    !pip install -U kubernetes==9.0.0
    !pip install -U azure

In [1]:
    !pip uninstall -y mlrun
    !pip install git+https://github.com/mlrun/mlrun.git@development
    !pip install -U git+https://github.com/yjb-ds/functions-demo.git

Uninstalling mlrun-0.4.1:
  Successfully uninstalled mlrun-0.4.1
  Cloning https://github.com/mlrun/mlrun.git (to revision development) to /tmp/pip-req-build-escg07rc
Branch development set up to track remote branch development from origin.
Switched to a new branch 'development'
  Running setup.py bdist_wheel for mlrun ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-dbpmdcil/wheels/ce/82/2f/a98d204a5dd1b27fa2a685cd11e705f1690d8f7ce2d8c08c9a
Successfully built mlrun
  Cloning https://github.com/yjb-ds/functions-demo.git to /tmp/pip-req-build-55keye6e
  Running setup.py bdist_wheel for functions ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-jxdio3ab/wheels/2b/c7/ec/2c6ddbe3a11b59aa979259c6f2a12f1c7b8203c9e4c844ebe5
Successfully built functions
  Found existing installation: functions 0.1.0
    Can't uninstall 'functions'. No files were found to uninstall.


<a id="nuclio"></a>
_______________________________________________________________________________

## **nuclio code section**

If you have never run nuclio functions in your notebooks, please uncomment and run the following:

    !pip install nuclio-jupyter

In [6]:
# nuclio: ignore
import nuclio

Install the following packages so they are available to the function:

In [7]:
%%nuclio cmd -c
pip install kfserving==0.2.0
pip install numpy==1.16.4 
pip install tensorflow==2.0.0b1
pip install pandas==0.25.3
pip install azure
pip install joblib
pip install git+https://github.com/mlrun/mlrun.git@development
pip install git+https://github.com/yjb-ds/functions-demo.git

In [8]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import kfserving
import numpy as np
import joblib
from typing import List
from tensorflow.keras.models import Sequential

from functions.models import (model_gen,
                              FeaturesEngineer,
                              pipeline_load)

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline


class MyKerasClassifier(kfserving.KFModel):
    def __init__(self,
                 name: str,
                 model_dir: str,
                 classifier: Sequential = None):
        """TFKerasClassifier
        
        Server model wrapper.
        
        :param name:            model name
        :param model_dir:       path of stored model
        :param classifier:      class type of classifier model
        
        """
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if classifier:
            self.ready = True

    def load(self):
        """Load model from storage.
        
        Note that loading to load a model we specify only a folder. Our custom
        loader takes care of the details.
        """
        self.classifier = pipeline_load(self.model_dir)
    
    def predict(self, body):
        """Generate model predictions from sample.
        
        :param body: A list of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

ModuleNotFoundError: No module named 'functions'

In [ ]:
# nuclio: end-code

<a id="deploy"></a>
_______________________________________________________________________________

## **deploy**

In [ ]:
from mlrun import new_model_server, mount_v3io

In [10]:
TARGET_PATH = "/User/mlrun/simdata"
MODEL_NAME = "tfkeras"

In [11]:
fn = new_model_server(MODEL_NAME, 
                      models={"tfkeras_joblib": TARGET_PATH}, 
                      model_class="MyKerasClassifier").apply(mount_v3io())

While debugging your project code, you may want to set the following flag to `True` in order to ensure that all the layers in your image get rebuilt and your changes included:

In [12]:
fn.spec.no_cache=True

After running the following cell, you can start watching the function's deployment by selecting **Projects** in the Iguazio platform UI, and clicking on the project name, and selecting the correct function and version:

In [ ]:
addr = fn.deploy(project=("mlrun-demos"))

[mlrun] 2020-01-04 16:13:35,635 deploy started
[nuclio] 2020-01-04 16:13:35,662 project name not found created new (mlrun-demos)


<a id="deploy"></a>
_______________________________________________________________________________

## **test**

In [1]:
import pandas as pd
import requests
import json

#### grab 2 rows of data - balanced dataset

In [2]:
features = pd.read_csv("x_test_50.csv").iloc[:2,:]
labels = pd.read_csv("y_test_50.csv").iloc[:2,:]

#### create an event and wrap it in json

In [3]:
event = {"instances" : features.values.tolist()}

If the notebook is restarted for some reason, however the function has already been deployed, simply uncomment the following cell and paste in the original endpoint here.  You can 
retrieve the function's enpoint address from the platform ui under **Projects**. Look for
the project name you gave to the deployment of interest, in our case **mlrun-demos**.
Click on the function in the project, copy the 'Invocation URL' and paste here:

In [4]:
# addr = "3.137.70.243:31811"

In [5]:
resp = requests.post(addr + "/tfkeras_joblib/predict", json=event)

json.loads(resp.content)

NameError: name 'addr' is not defined